Interactive Map for North American UFO Sightings

In [1]:
import pandas as pd
import folium
from folium import plugins


df = pd.read_csv('ufo_data_nuforc.csv') 

# Ensure that 'lat' and 'lng' columns are numeric (if not, you may need to convert them)
df['lat'] = pd.to_numeric(df['lat'], errors='coerce')
df['lng'] = pd.to_numeric(df['lng'], errors='coerce')

# Create a base map centered at an initial location
m = folium.Map(location=[df['lat'].mean(), df['lng'].mean()], zoom_start=5, tiles='cartodb positron')

# Create a marker cluster for better performance
marker_cluster = plugins.MarkerCluster(name='UFO Sightings Cluster').add_to(m)

# Calculate sighting intensity for the color gradient
max_intensity = df.shape[0]
df['intensity'] = 100 * df.groupby(['lat', 'lng'])['date'].transform('count') / max_intensity

# Add data points to the map with tooltips and color gradient
for index, row in df.iterrows():
    folium.CircleMarker(
        location=[row['lat'], row['lng']],
        radius=5,
        fill=True,
        fill_color=f"#{int(row['intensity']):02X}0000",  # Adjust the color based on intensity
        color='blue',
        fill_opacity=0.7,
        popup=f"<b>Date:</b> {row['date']}<br><b>City:</b> {row['city']}<br><b>Shape:</b> {row['shape']}",
    ).add_to(marker_cluster)

# Add a color gradient legend
colormap = plugins.MiniMap(
    gradient={0: 'blue', max_intensity / 2: 'green', max_intensity: 'red'},
    caption='Sighting Intensity (%)'
)
colormap.add_to(m)

# Add a layer control
folium.LayerControl(collapsed=False).add_to(m)

# Display the map
m.save('fancy_ufo_sightings_heatmap.html')  # Save the map as an HTML file
m  # Display the map in a Jupyter Notebook (if using a Jupyter environment)
